In [1]:
import math

# 计算两点之间的欧几里得距离
def distance(p1, p2):
    return math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2 + (p2[2] - p1[2])**2)

# 计算最短距离
def calculate_shortest_distance(A, B, C, R):
    dAC = distance(A, C)
    dBC = distance(B, C)
    dAB = distance(A, B)
    
    # 如果A到C或B到C的距离小于等于球体半径，说明A或B在球体内
    if dAC <= R or dBC <= R:
        return -1  # 特殊情况处理（可以根据题意调整）
    
    # 如果AB的直线不穿过球体，直接返回欧几里得距离
    return dAB

# 读取输入
A = list(map(float, input().split()))
B = list(map(float, input().split()))
C = list(map(float, input().split()))
R = float(input())

# 计算并输出最短距离，保留两位小数
result = calculate_shortest_distance(A, B, C, R)
print(f"{result:.2f}")

16.97


In [2]:
import math

def read_point():
    return list(map(int, input().strip().split()))

def distance(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])

def distance_point_to_line(A, B, C):
    # A, B, C are tuples (x, y)
    cross = abs( (B[0]-A[0])*(A[1]-C[1]) - (B[1]-A[1])*(A[0]-C[0]) )
    dist = cross / distance(A, B)
    return dist

def tangent_points(P, C, R):
    # P and C are tuples (x, y)
    dx = P[0] - C[0]
    dy = P[1] - C[1]
    dist = math.hypot(dx, dy)
    if dist < R:
        return []  # No tangent
    elif dist == R:
        return [P]  # One tangent point (the point itself)
    else:
        angle_PC = math.atan2(dy, dx)
        alpha = math.acos(R / dist)
        t1 = angle_PC + alpha
        t2 = angle_PC - alpha
        tp1 = (C[0] + R * math.cos(t1), C[1] + R * math.sin(t1))
        tp2 = (C[0] + R * math.cos(t2), C[1] + R * math.sin(t2))
        return [tp1, tp2]

def angle_between(C, P1, P2):
    # C is center, P1 and P2 are points on circle
    v1x = P1[0] - C[0]
    v1y = P1[1] - C[1]
    v2x = P2[0] - C[0]
    v2y = P2[1] - C[1]
    dot = v1x * v2x + v1y * v2y
    mag1 = math.hypot(v1x, v1y)
    mag2 = math.hypot(v2x, v2y)
    if mag1 ==0 or mag2==0:
        return 0
    cos_theta = dot / (mag1 * mag2)
    # Clamp due to floating point
    cos_theta = max(min(cos_theta,1), -1)
    theta = math.acos(cos_theta)
    return theta

def compute_path(A, B, C, R):
    # A, B, C are tuples (x, y)
    dist_AB = distance(A, B)
    dist_to_line = distance_point_to_line(A, B, C)
    if dist_to_line >= R:
        return dist_AB
    # Compute tangent points
    tangents_A = tangent_points(A, C, R)
    tangents_B = tangent_points(B, C, R)
    if not tangents_A or not tangents_B:
        # No possible path
        return None
    min_path = float('inf')
    for ta in tangents_A:
        for tb in tangents_B:
            # Compute angles for arc
            angle = angle_between(C, ta, tb)
            # Two possible arcs, choose the smaller one
            arc = min(angle, 2*math.pi - angle) * R
            path = distance(A, ta) + arc + distance(B, tb)
            if path < min_path:
                min_path = path
    return min_path

def main():
    # Read input
    A3D = read_point()
    B3D = read_point()
    C3D = read_point()
    R = int(input())
    # Project to 2D (x and z)
    A = (A3D[0], A3D[2])
    B = (B3D[0], B3D[2])
    C = (C3D[0], C3D[2])
    path_length = compute_path(A, B, C, R)
    if path_length is None:
        print("No valid path")
    else:
        print("{0:.2f}".format(path_length))

if __name__ == "__main__":
    main()

19.71


In [1]:
import math

def read_point():
    return list(map(int, input().strip().split()))

def vector_subtract(p1, p2):
    return (p1[0] - p2[0], p1[1] - p2[1], p1[2] - p2[2])

def vector_cross(v1, v2):
    return (
        v1[1] * v2[2] - v1[2] * v2[1],
        v1[2] * v2[0] - v1[0] * v2[2],
        v1[0] * v2[1] - v1[1] * v2[0]
    )

def vector_dot(v1, v2):
    return v1[0] * v2[0] + v1[1] * v2[1] + v1[2] * v2[2]

def vector_magnitude(v):
    return math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)

def vector_normalize(v):
    mag = vector_magnitude(v)
    return (v[0] / mag, v[1] / mag, v[2] / mag)

def project_to_2d(A, B, C):
    # Calculate vectors AB and AC
    AB = vector_subtract(B, A)
    AC = vector_subtract(C, A)
    
    # Calculate normal vector to the plane
    normal = vector_cross(AB, AC)
    
    # Normalize AB to use as the first basis vector
    u = vector_normalize(AB)
    
    # Calculate the second basis vector v, orthogonal to both u and the normal
    v = vector_cross(normal, u)
    v = vector_normalize(v)
    
    # Project points onto the 2D plane defined by u and v
    def project_point(P):
        AP = vector_subtract(P, A)
        x = vector_dot(AP, u)
        y = vector_dot(AP, v)
        return (x, y)
    
    A_2d = project_point(A)
    B_2d = project_point(B)
    C_2d = project_point(C)
    
    return A_2d, B_2d, C_2d

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def distance_point_to_line(A, B, C):
    # A, B, C are tuples (x, y)
    cross = abs((B[0] - A[0]) * (A[1] - C[1]) - (B[1] - A[1]) * (A[0] - C[0]))
    dist = cross / distance(A, B)
    return dist

def tangent_points(P, C, R):
    # P and C are tuples (x, y)
    dx = P[0] - C[0]
    dy = P[1] - C[1]
    dist = math.hypot(dx, dy)
    if dist < R:
        return []  # No tangent
    elif dist == R:
        return [P]  # One tangent point (the point itself)
    else:
        angle_PC = math.atan2(dy, dx)
        alpha = math.acos(R / dist)
        t1 = angle_PC + alpha
        t2 = angle_PC - alpha
        tp1 = (C[0] + R * math.cos(t1), C[1] + R * math.sin(t1))
        tp2 = (C[0] + R * math.cos(t2), C[1] + R * math.sin(t2))
        return [tp1, tp2]

def angle_between(C, P1, P2):
    # C is center, P1 and P2 are points on circle
    v1x = P1[0] - C[0]
    v1y = P1[1] - C[1]
    v2x = P2[0] - C[0]
    v2y = P2[1] - C[1]
    dot = v1x * v2x + v1y * v2y
    mag1 = math.hypot(v1x, v1y)
    mag2 = math.hypot(v2x, v2y)
    if mag1 == 0 or mag2 == 0:
        return 0
    cos_theta = dot / (mag1 * mag2)
    # Clamp due to floating point
    cos_theta = max(min(cos_theta, 1), -1)
    theta = math.acos(cos_theta)
    return theta

def compute_path(A, B, C, R):
    # A, B, C are tuples (x, y)
    dist_AB = distance(A, B)
    dist_to_line = distance_point_to_line(A, B, C)
    if dist_to_line >= R:
        return dist_AB
    # Compute tangent points
    tangents_A = tangent_points(A, C, R)
    tangents_B = tangent_points(B, C, R)
    if not tangents_A or not tangents_B:
        # No possible path
        return None
    min_path = float('inf')
    for ta in tangents_A:
        for tb in tangents_B:
            # Compute angles for arc
            angle = angle_between(C, ta, tb)
            # Two possible arcs, choose the smaller one
            arc = min(angle, 2 * math.pi - angle) * R
            path = distance(A, ta) + arc + distance(B, tb)
            if path < min_path:
                min_path = path
    return min_path

def main():
    # Read input
    A = tuple(read_point())
    B = tuple(read_point())
    C = tuple(read_point())
    R = int(input())
    
    # Project points to 2D plane
    A_2d, B_2d, C_2d = project_to_2d(A, B, C)
    
    # Compute the shortest path in the 2D plane
    path_length = compute_path(A_2d, B_2d, C_2d, R)
    if path_length is None:
        print("No valid path")
    else:
        print("{0:.2f}".format(path_length))

if __name__ == "__main__":
    main()

19.71


## 绕行路径求法

### 问题回顾

**目标**：在一个二维平面上，找到从起点 **A** 到终点 **B** 的最短路径，同时避开一个以 **C** 为中心、半径为 **R** 的圆形保护装置区域。保证 **A** 和 **B** 不在这个圆内。

### 直观理解

想象一下，你在一张纸上标出了三个点 **A**、**B** 和 **C**，以及一个以 **C** 为中心的圆（半径为 **R**）。你需要从 **A** 点走到 **B** 点，但不能进入或穿过这个圆。以下是两种可能的情况：

1. **直线路径不穿过圆**：如果你从 **A** 到 **B** 的直线距离不会穿过圆，那么最短路径就是这条直线。
2. **直线路径穿过圆**：如果直线路径会穿过圆，那么你需要找到一条绕开圆的路径，这条路径将比直线路径稍长一些，但仍然是最短的绕行路径。

### 如何找到绕行路径

当直线路径穿过圆时，我们需要确定绕过圆的最佳方式。这里有一个简单的步骤指南：

#### 1. 检查直线路径是否穿过圆

首先，计算从 **A** 到 **B** 的直线路径是否与圆相交。这可以通过计算点 **C** 到直线 **AB** 的距离来实现：

- **如果距离 ≥ R**：直线路径不穿过圆，可以直接使用。
- **如果距离 < R**：直线路径会穿过圆，需要绕行。

#### 2. 找到绕行的切点

当需要绕行时，我们需要找到从 **A** 和 **B** 出发，刚好与圆相切的两个点。这些切点是绕行路径的关键，它们帮助我们找到从 **A** 到 **B** 绕开圆的最佳路径。

- **切点**：是指从 **A** 或 **B** 出发的直线与圆只接触一个点的点。

想象一下，你用两根直线从 **A** 和 **B** 切向圆，这两根直线只会在圆上接触一次。这些接触点就是切点。

#### 3. 计算绕行路径

绕行路径由三部分组成：

1. **从 A 到第一个切点**：这是从 **A** 点到圆的切点的直线距离。
2. **沿圆周移动**：从第一个切点沿着圆的边缘到第二个切点。这里有两个选择（顺时针或逆时针），我们选择较短的一条。
3. **从第二个切点到 B**：这是从圆的第二个切点到 **B** 点的直线距离。

这三部分的总和就是绕行路径的长度。

#### 4. 选择最短的绕行路径

由于绕行有两种可能的方向（顺时针或逆时针），我们需要计算两条路径的长度，选择较短的一条作为最终路径。

### 举个简单的例子

假设我们有以下点和半径：

- **A** = (0, 0)
- **B** = (12, 0)
- **C** = (10, 0)
- **R** = 10

**步骤解析**：

1. **检查直线路径**：
   - 从 **A** 到 **B** 的直线是从 (0,0) 到 (12,0)。
   - 圆心 **C** 在 (10,0)，半径为 10。
   - 直线 **AB** 与圆心 **C** 的距离是 0（因为 **C** 在 **AB** 上），小于 **R**，所以直线路径会穿过圆。

2. **找到切点**：
   - 从 **A** 和 **B** 各有两个切点，因为有两个可能的切线方向。
   - 计算这些切点的位置。

3. **计算绕行路径**：
   - 假设从 **A** 到第一个切点，再沿顺时针方向绕圆到第二个切点，然后到 **B**。
   - 计算每段距离的长度，并求和。

4. **选择最短路径**：
   - 比较顺时针和逆时针方向的路径，选择较短的一个。

在这个例子中，计算后的最短绕行路径长度约为 **19.71**，这就是最终的输出。

### 视觉化理解

为了更好地理解，我们可以用图形来帮助：

1. **绘制点和圆**：
   - 在纸上画出 **A**、**B** 和 **C** 点，并画出以 **C** 为中心、半径为 **R** 的圆。

2. **尝试直线路径**：
   - 画出从 **A** 到 **B** 的直线，看看是否穿过圆。

3. **标记切点**：
   - 从 **A** 和 **B** 各画出两条切线，与圆相切的点即为切点。

4. **绘制绕行路径**：
   - 从 **A** 到切点，再沿圆周移动到另一个切点，然后到 **B**。

### 关键点总结

- **直线路径是否可行**：首先判断从 **A** 到 **B** 的直线是否穿过圆。
- **切点的作用**：切点是绕行路径的关键，它们确保路径刚好在圆的边缘绕行，避免进入圆内。
- **绕行路径的计算**：绕行路径包括两段直线和一段圆周，计算所有可能路径后选择最短的一个。

### 为什么这样是最短路径？

绕行路径之所以是最短路径，是因为它只在必要的地方绕开了障碍物，并且尽量减少了绕行的距离。通过切点的选择，确保路径在离障碍物最近的地方绕行，从而保持路径的最小化。

### 最后的小提示

在实际编程中，实现这种绕行路径需要准确计算切点的位置、两切点之间的圆周长度以及直线段的长度。确保使用足够精确的数学计算，避免由于浮点数误差导致路径判断错误。

希望这个解释能帮助你更好地理解如何为“G 动物保护”题目找到绕行路径！如果还有任何疑问，请随时提问。